In [2]:
import os
import json
import pickle
from bisect import bisect_left, bisect_right
from datetime import datetime, timedelta
from collections import defaultdict, Counter

import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# GPU hack if you need
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Data

Columns
- `party_rk` – client unique identifier
- `account_rk` – client account unique identifier
- `financial_account_type_cd` – debit/credit card flag
- `transaction_dttm` – operation datetime
- `transaction_type_desc` – purchase/payment/...
- `transaction_amt_rur` – transaction price
- `merchant_type` - DUTY FREE STORES/FUEL DEALERS/RESTAURANTS/ etc
- `merchant_group_rk` - McDonald's/Wildberries/ etc

It's important that table is already sorted by `transaction_dttm` column!!!

In [11]:
DATADIR = "./hackathon_data" # "./data"
transactions_path = f"{DATADIR}/avk_hackathon_data_transactions.csv"
data = pd.read_csv(f"{DATADIR}/avk_hackathon_data_transactions.csv")

In [15]:
data_balance = pd.read_csv(f"{DATADIR}/avk_hackathon_data_account_x_balance.csv")

In [5]:
data_soc = pd.read_csv(f"{DATADIR}/avk_hackathon_data_party_x_socdem.csv")

## Mappings
~1 min

In [3]:
# Prepare & save mappings
mappings = defaultdict(dict)
unk_token = "<UNK>"

def create_mapping(values):
    mapping = {unk_token: 0}
    for v in values:
        if not pd.isna(v):
            mapping[str(v)] = len(mapping)

    return mapping


for col in tqdm(
    [
        "transaction_type_desc",
        "merchant_rk",
        "merchant_type",
        "merchant_group_rk",
        "category",
        "financial_account_type_cd",
    ]
):

    col_values = (
        pd.read_csv(transactions_path, usecols=[col])[col]
        .fillna(unk_token)
        .astype(str)
    )
    mappings[col] = create_mapping(col_values.unique())
    del col_values


with open(f"{DATADIR}/mappings.json", "w") as f:
    json.dump(mappings, f)

  0%|          | 0/6 [00:05<?, ?it/s]


KeyboardInterrupt: 

In [12]:
# load mappings
unk_token = "<UNK>"
with open(f"{DATADIR}/mappings.json", 'r') as f:
     mappings = json.load(f)

In [9]:
mappings

{'transaction_type_desc': {'<UNK>': 0,
  'Покупка': 1,
  'Оплата услуг': 2,
  'Платеж': 3,
  'Снятие наличных': 4},
 'merchant_rk': {'<UNK>': 4816,
  '88676.0': 1,
  '887248.0': 2,
  '830014.0': 3,
  '363834.0': 4,
  '85919.0': 5,
  '146084.0': 6,
  '166649.0': 7,
  '1117458.0': 8,
  '798183.0': 9,
  '123479.0': 10,
  '362659.0': 11,
  '179531.0': 12,
  '988289.0': 13,
  '467110.0': 14,
  '596938.0': 15,
  '1116811.0': 16,
  '226559.0': 17,
  '528509.0': 18,
  '1097997.0': 19,
  '415888.0': 20,
  '500918.0': 21,
  '149715.0': 22,
  '516518.0': 23,
  '439054.0': 24,
  '245107.0': 25,
  '521220.0': 26,
  '166902.0': 27,
  '114006.0': 28,
  '611670.0': 29,
  '567084.0': 30,
  '55980.0': 31,
  '250736.0': 32,
  '93298.0': 33,
  '1092595.0': 34,
  '46122.0': 35,
  '552790.0': 36,
  '374309.0': 37,
  '695981.0': 38,
  '501347.0': 39,
  '261009.0': 40,
  '372350.0': 41,
  '3620.0': 42,
  '560151.0': 43,
  '651570.0': 44,
  '1059424.0': 45,
  '774130.0': 46,
  '118933.0': 47,
  '356081.0': 48,

## Parse transactions by users
~ 40 min

In [19]:
party2cat = defaultdict(list)
usecols = [
    "party_rk",
    "category",
]

for chunk in tqdm(
    pd.read_csv(transactions_path, usecols=usecols, chunksize=100_000w)
):
    chunk["category"] = (
        chunk["category"].fillna(unk_token).astype(str)
    )
    print(chunk["category"])  
    for i, row in chunk.iterrows():
        party2cat[row.party_rk].append(
            mappings["category"][row.category]
        )

    del chunk
        
pickle.dump(party2cat, open(f"{DATADIR}/party2cat.pkl", "wb"))

9it [00:00, 81.85it/s]

0               Сувениры
1               Фаст Фуд
2           Супермаркеты
3             Дом/Ремонт
4           Супермаркеты
5           Супермаркеты
6           Супермаркеты
7       Сервисные услуги
8           Супермаркеты
9                Красота
10                 <UNK>
11          Супермаркеты
12         Разные товары
13             Транспорт
14                 <UNK>
15                 <UNK>
16    Медицинские услуги
17          Супермаркеты
18               Топливо
19          Одежда/Обувь
20               Красота
21          Одежда/Обувь
22          Супермаркеты
23              Наличные
24          Супермаркеты
25          Одежда/Обувь
26          Супермаркеты
27              Фаст Фуд
28             Транспорт
29          Супермаркеты
             ...        
70         Разные товары
71              Наличные
72          Супермаркеты
73     Финансовые услуги
74          Супермаркеты
75               Топливо
76          Супермаркеты
77          Супермаркеты
78          Супермаркеты


27it [00:00, 82.56it/s]

1800          Супермаркеты
1801              Фаст Фуд
1802          Супермаркеты
1803                 <UNK>
1804          Супермаркеты
1805          Супермаркеты
1806          Супермаркеты
1807     Финансовые услуги
1808    Медицинские услуги
1809         Разные товары
1810          Супермаркеты
1811                Аптеки
1812              Фаст Фуд
1813          Супермаркеты
1814          Супермаркеты
1815              Фаст Фуд
1816              Фаст Фуд
1817              Фаст Фуд
1818         Разные товары
1819          Супермаркеты
1820             Транспорт
1821          Супермаркеты
1822                 <UNK>
1823          Супермаркеты
1824                 <UNK>
1825             Рестораны
1826              Наличные
1827          Супермаркеты
1828              Наличные
1829          Супермаркеты
               ...        
1870            Дом/Ремонт
1871     Финансовые услуги
1872              Наличные
1873     Финансовые услуги
1874                 <UNK>
1875                 <UNK>
1

45it [00:00, 82.86it/s]

...

711it [00:08, 86.15it/s]

70100            Транспорт
70101         Супермаркеты
70102             Наличные
70103           Автоуслуги
70104        Разные товары
70105        Разные товары
70106               Аптеки
70107         Супермаркеты
70108        Разные товары
70109         Одежда/Обувь
70110                  НКО
70111                <UNK>
70112         Супермаркеты
70113         Супермаркеты
70114              Красота
70115               Аптеки
70116         Супермаркеты
70117              Топливо
70118         Супермаркеты
70119         Супермаркеты
70120                <UNK>
70121        Разные товары
70122         Супермаркеты
70123         Супермаркеты
70124         Супермаркеты
70125            Рестораны
70126        Разные товары
70127         Супермаркеты
70128                <UNK>
70129             Фаст Фуд
               ...        
70170    Финансовые услуги
70171                <UNK>
70172                <UNK>
70173         Супермаркеты
70174                <UNK>
70175                <UNK>
7

721it [00:08, 85.94it/s]

71900           Транспорт
71901        Супермаркеты
71902        Супермаркеты
71903          Дом/Ремонт
71904           Рестораны
71905          Дом/Ремонт
71906        Супермаркеты
71907               <UNK>
71908           Транспорт
71909        Супермаркеты
71910          Автоуслуги
71911           Транспорт
71912             Топливо
71913        Супермаркеты
71914        Одежда/Обувь
71915           Транспорт
71916        Супермаркеты
71917             Топливо
71918        Супермаркеты
71919        Супермаркеты
71920        Одежда/Обувь
71921       Разные товары
71922       Связь/Телеком
71923        Супермаркеты
71924    Сервисные услуги
71925        Супермаркеты
71926           Транспорт
71927        Супермаркеты
71928        Супермаркеты
71929        Супермаркеты
               ...       
71970           Транспорт
71971        Супермаркеты
71972    Сервисные услуги
71973           Рестораны
71974        Супермаркеты
71975           Транспорт
71976            Фаст Фуд
71977       

KeyboardInterrupt: 

In [9]:
# Prepare & save client data
party2dates = defaultdict(list)  # for each party save a series of the transaction dates 
party2sum = defaultdict(list)  # for each party save a series of the transaction costs 
party2merchant_type = defaultdict(list)  # for each party save a series of the transaction_type 
party2trans_type = defaultdict(list)  # for each party save a series of the transaction merchant_type

usecols = [
    "party_rk",
    "transaction_dttm",
    "transaction_amt_rur",
    "merchant_type",
    "transaction_type_desc",
]

for chunk in tqdm(
    pd.read_csv(transactions_path, usecols=usecols, chunksize=100_000)
):

    chunk["merchant_type"] = (
        chunk["merchant_type"].fillna(unk_token).astype(str)
    )
    chunk["transaction_type_desc"] = (
        chunk["transaction_type_desc"].fillna(unk_token).astype(str)
    )
    chunk["transaction_amt_rur"] = chunk["transaction_amt_rur"].fillna(0)

    for i, row in chunk.iterrows():
        party2dates[row.party_rk].append(row.transaction_dttm)
        party2sum[row.party_rk].append(row.transaction_amt_rur)
        party2merchant_type[row.party_rk].append(
            mappings["merchant_type"][row.merchant_type]
        )
        party2trans_type[row.party_rk].append(
            mappings["transaction_type_desc"][row.transaction_type_desc]
        )

    del chunk

pickle.dump(party2dates, open(f"{DATADIR}/party2dates.pkl", "wb"))
pickle.dump(party2sum, open(f"{DATADIR}/party2sum.pkl", "wb"))
pickle.dump(party2merchant_type, open(f"{DATADIR}/party2merchant_type.pkl", "wb"))
pickle.dump(party2trans_type, open(f"{DATADIR}/party2trans_type.pkl", "wb"))

0it [00:03, ?it/s]


KeyboardInterrupt: 

In [13]:
# load client data
party2dates = pickle.load(open(f"{DATADIR}/party2dates.pkl", 'rb'))
party2sum = pickle.load(open(f"{DATADIR}/party2sum.pkl", 'rb'))
party2merchant_type = pickle.load(open(f"{DATADIR}/party2merchant_type.pkl", 'rb'))
party2trans_type = pickle.load(open(f"{DATADIR}/party2trans_type.pkl", 'rb'))
party2cat = pickle.load(open(f"{DATADIR}/party2cat.pkl", 'rb'))

In [30]:
categories_types = mappings["category"]
categories_types

{'<UNK>': 7,
 'Сувениры': 1,
 'Фаст Фуд': 2,
 'Супермаркеты': 3,
 'Дом/Ремонт': 4,
 'Сервисные услуги': 5,
 'Красота': 6,
 'Разные товары': 7,
 'Транспорт': 8,
 'Медицинские услуги': 9,
 'Топливо': 10,
 'Одежда/Обувь': 11,
 'Наличные': 12,
 'Связь/Телеком': 13,
 'Частные услуги': 14,
 'Финансовые услуги': 15,
 'Рестораны': 16,
 'Развлечения': 17,
 'НКО': 18,
 'Книги': 19,
 'Кино': 20,
 'Автоуслуги': 21,
 'Музыка': 22,
 'Отели': 23,
 'Аптеки': 24,
 'Цветы': 25,
 'Ж/д билеты': 26,
 'Авиабилеты': 27,
 'Спорттовары': 28,
 'Госсборы': 29,
 'Аренда авто': 30,
 'Животные': 31,
 'Duty Free': 32,
 'Турагентства': 33,
 'Образование': 34,
 'Искусство': 35,
 'Фото/Видео': 36}

# Decision process

### Idea

As we had predictions of transactions, which can be performed by a particular person. So we decided to predict income/outcome of a person in that months in order to make recommendation. Based on that information we can suggest to him/her to participate in different kinds of challenges and quests.

### Investement suggestions
either to make an investement and buy shares, create an "automatic savings plan" or to make a personal plan of spending money, make a suggestion of how to spend less.

### Entertainment suggestions
Get quests and challenges for your favourite time spending. Cinemas, concerts and restaurants. 

In [6]:
mappings_cat = defaultdict(dict)
unk_token = "<UNK>"

for name in mappings['merchant_type'].keys():
    if name == unk_token:
        continue
    else:
        mappings_cat[mappings['merchant_type'][name]] = list()

col_values = (
        pd.read_csv(transactions_path, usecols=["merchant_type", 'category'])
        .fillna(unk_token)
        .astype(str)
    )

for i, row in tqdm(
    col_values.iterrows()
    ):
    
    if mappings["category"][row.category] not in mappings_cat[mappings['merchant_type'][row.merchant_type]]:
        mappings_cat[mappings['merchant_type'][row.merchant_type]].append(mappings["category"][row.category])
        
with open(f"{DATADIR}/mappings_cat_merchant_type.json", "w") as f:
    json.dump(mappings_cat, f)

11987617it [15:59, 12488.79it/s]


In [45]:
# load mappings
unk_token = "<UNK>"
with open(f"{DATADIR}/mappings_cat_merchant_type.json", 'r') as f:
     mappings_cat = json.load(f)

In [46]:
# if person is interested in money increasing/saving strategies
money_recommendations_pack = {
    'Capital saving strategy': {
        'Expenses decrease suggestion.': ' https://journal.tinkoff.ru/selected/budget/  ',
        'Card limits setting.': '  https://help.tinkoff.ru/drive-dc/limit-add-card/add-card-set-limits/  ',
    },
    'Safe capital increase strategies': {
        'Deposits suggestion.': '  https://www.tinkoff.ru/deposit/  ',
        'Charity suggestion.': {
            'Moscow': '  https://www.tinkoff.ru/payments/categories/blagotvoritelnost/  ',
            'All': '  https://www.tinkoff.ru/about/news/04082020-tinkoff-launched-charity-service/  ',
        },
        'Automatic savings.': '  https://www.tinkoff.ru/eng/media/news/08062020-investments-box-eng/  ',
    },
    'Capital increase strategies': {
        'Investement suggestion.': '  https://www.tinkoff.ru/invest/ ',
        'Charity suggestion.': {
            'Moscow': '  https://www.tinkoff.ru/payments/categories/blagotvoritelnost/ ',
            'All': '  https://www.tinkoff.ru/about/news/04082020-tinkoff-launched-charity-service/ ',
        },
    },
}

# if person is inetrested in entertainment
entertainment_recommendation_pack = {
    'Cinemas': {
        'Moscow': 'Visit three cinemas in Moscow. Get cashback for buying tickets at Tinkoff. https://www.tinkoff.ru/entertainment/moskva/movies/  ',
        "All": 'Visit three the oldest cinemas. Get cashback for buying tickets at Tinkoff.  https://help.tinkoff.ru/junior/earn/cashback-for-movie/ ',
    },
    'Restaurants': 'Visit three the most interesting Georgian restaurants.  https://www.tinkoff.ru/entertainment/:city/restaurants/ ',
    'Music': 'Visit three coolest rock-concerts in three months.  https://www.tinkoff.ru/entertainment/concerts/ ',
}

In [53]:
date_months = '2019-06'
user_num = 33943

def get_data_party(user_num, date_months):
    data_party = dict()
    
    data_user = data[data['party_rk'] == user_num]
    months = [item[:-3] for item in data_user['transaction_dttm']]
    data_user['month'] = months
    data_user = data_user[data_user['month'] == date_months]['merchant_type'].dropna()

    data_user_balance = data_balance[data_balance['party_rk'] == user_num]
    months = [item[:-3] for item in data_user_balance['cur_month']]
    data_user_balance['month'] = months
    data_user_balance = sum(data_user_balance[data_user_balance['month'] == date_months]['balance_chng'])

    city = data_soc[data_soc['party_rk'] == user_num]['region_flg']
    
    data_party["merchant_type"] = list(data_user)
    data_party["balance"] = data_user_balance
    data_party["city"] = np.int(city)
    
    return data_party

def get_key(d, value):
    for k, v in d.items():
        if v == value:
            return k
        

data_party = get_data_party(user_num, date_months)

In [58]:
def give_advise_category_based(cat, balance, city):
    
    # financial advises
    if balance < 0:
        strategy = 'Capital saving strategy'
        if cat in ['Развлечения', 'Рестораны']:
            return money_recommendations_pack[strategy]['Card limits setting.']
        else:
            return money_recommendations_pack[strategy]['Expenses decrease suggestion.']
    else:
        if balance >= 10000:
            strategy = 'Safe capital increase strategies'
            
            if city == 0: 
                city_name = 'Moscow'
                link = money_recommendations_pack[strategy]['Charity suggestion.'][city_name]
            else:
                city_name = 'All'
                link = money_recommendations_pack[strategy]['Charity suggestion.'][city_name]
                
            if cat in ['Развлечения', 'Кино']:
                link += '  '
                link += entertainment_recommendation_pack['Cinemas'][city_name]
        else:
            strategy = 'Capital increase strategies'
            return money_recommendations_pack[strategy]['Investement suggestion.']
        
    return link

In [54]:
def recommend_quest_for_user(data_party):
    
    categories_transac = np.zeros(len(mappings['category']))

    for name in data_party['merchant_type']:
        # get categories of transactions
        categories_transac[mappings_cat[str(mappings['merchant_type'][str(name)])][0]] += 1

    max_ind_category_1 = categories_transac.argmax()
    priority_cat_1 = get_key(mappings["category"], max_ind_category_1)
    categories_transac[max_ind_category_1] = 0

    max_ind_category_2 = categories_transac.argmax()
    priority_cat_2 = get_key(mappings["category"], max_ind_category_2)
    categories_transac[max_ind_category_2] = 0

    max_ind_category_3 = categories_transac.argmax()
    priority_cat_3 = get_key(mappings["category"], max_ind_category_3)
    categories_transac[max_ind_category_3] = 0

    max_ind_category_4 = categories_transac.argmax()
    priority_cat_4 = get_key(mappings["category"], max_ind_category_4)
    categories_transac[max_ind_category_3] = 0

    priorities = [priority_cat_1, priority_cat_2, priority_cat_3, priority_cat_4]
    
    
    return advises

In [60]:
def get_advise(data_party):
    categories_transac = np.zeros(len(mappings['category']))

    for name in data_party['merchant_type']:
        # get categories of transactions
        categories_transac[mappings_cat[str(mappings['merchant_type'][str(name)])][0]] += 1

    max_ind_category_1 = categories_transac.argmax()
    priority_cat_1 = get_key(mappings["category"], max_ind_category_1)
    categories_transac[max_ind_category_1] = 0

    max_ind_category_2 = categories_transac.argmax()
    priority_cat_2 = get_key(mappings["category"], max_ind_category_2)
    categories_transac[max_ind_category_2] = 0

    max_ind_category_3 = categories_transac.argmax()
    priority_cat_3 = get_key(mappings["category"], max_ind_category_3)
    categories_transac[max_ind_category_3] = 0

    max_ind_category_4 = categories_transac.argmax()
    priority_cat_4 = get_key(mappings["category"], max_ind_category_4)
    categories_transac[max_ind_category_3] = 0

    priorities = [priority_cat_1, priority_cat_2, priority_cat_3, priority_cat_4]

    advise_1 = give_advise_category_based(priority_cat_1, data_party['balance'], data_party['city'])
    return advise_1

In [62]:
# The model can give an advise what can person do with their financies in the next month or can plan activities
get_advise(data_party)

'  https://www.tinkoff.ru/payments/categories/blagotvoritelnost/  '